## This script accesses the CESM2 Soil Moisture data for the Scenario years from GLADE repository of NCAR

In [1]:
import xarray as xr
from netCDF4 import Dataset 
import numpy as np
from numpy import unravel_index

from matplotlib import pyplot as plt 
from numpy import linspace 


In [2]:
# Projection
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [3]:
from descartes import PolygonPatch
import shapefile as shp  # Requires the pyshp package
import cartopy.io.shapereader as shpreader

In [4]:
# Common imports
import calendar
import datetime
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from numpy import array, ma
import scipy.io

In [5]:
cd '/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/month_1/SOILWATER_10CM'

/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/month_1/SOILWATER_10CM


In [6]:
pwd

'/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/month_1/SOILWATER_10CM'

## Accessing One File

In [7]:
filePath = '/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/month_1/SOILWATER_10CM'
fileName_Sample = 'b.e21.BHISTcmip6.f09_g17.LE2-1001.001.clm2.h0.SOILWATER_10CM.185001-185912.nc'
ncFile_soilMoist_Sample = str(filePath) + '/' + str(fileName_Sample)
print(ncFile_soilMoist_Sample)

/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/month_1/SOILWATER_10CM/b.e21.BHISTcmip6.f09_g17.LE2-1001.001.clm2.h0.SOILWATER_10CM.185001-185912.nc


In [8]:
file_xr_Sample  = xr.open_dataset(ncFile_soilMoist_Sample) 
file_xr_Sample

<xarray.Dataset>
Dimensions:         (hist_interval: 2, lat: 192, levdcmp: 25, levgrnd: 25, levlak: 10, lon: 288, time: 120)
Coordinates:
  * levgrnd         (levgrnd) float32 0.01 0.04 0.09 ... 28.870724 41.998436
  * levlak          (levlak) float32 0.05 0.6 2.1 4.6 ... 25.6 34.325 44.775
  * levdcmp         (levdcmp) float32 0.01 0.04 0.09 ... 28.870724 41.998436
  * lon             (lon) float32 0.0 1.25 2.5 3.75 ... 356.25 357.5 358.75
  * lat             (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * time            (time) datetime64[ns] 1850-02-01 1850-03-01 ... 1860-01-01
Dimensions without coordinates: hist_interval
Data variables:
    area            (lat, lon) float32 ...
    landfrac        (lat, lon) float32 ...
    landmask        (lat, lon) float64 ...
    pftmask         (lat, lon) float64 ...
    nbedrock        (lat, lon) float64 ...
    ZSOI            (levgrnd, lat, lon) float32 ...
    DZSOI           (levgrnd, lat, lon) float32 ...
    WATSAT     

## Find the unit

In [9]:
param_sm = file_xr_Sample['SOILWATER_10CM']
print(param_sm.shape)
print(param_sm)

(120, 192, 288)
<xarray.DataArray 'SOILWATER_10CM' (time: 120, lat: 192, lon: 288)>
[6635520 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.25 357.5 358.75
  * lat      (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * time     (time) datetime64[ns] 1850-02-01 1850-03-01 ... 1860-01-01
Attributes:
    long_name:     soil liquid water + ice in top 10cm of soil (veg landunits...
    units:         kg/m2
    cell_methods:  time: mean


In [10]:
file_soilMoist_Sample = Dataset(ncFile_soilMoist_Sample, mode='r')
file_soilMoist_Sample

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    title: CLM History file information
    comment: NOTE: None of the variables are weighted by land fraction!
    Conventions: CF-1.0
    history: created on 01/31/20 16:34:57
    source: Community Land Model CLM4.0
    hostname: aleph
    username: sunseon
    version: cesm2.1.2-rc.03-18-g4ee0b23
    revision_id: $Id: histFileMod.F90 42903 2012-12-21 15:32:10Z muszala $
    case_title: UNSET
    case_id: b.e21.BHISTcmip6.f09_g17.LE2-1001.001
    Surface_dataset: surfdata_0.9x1.25_hist_78pfts_CMIP6_simyr1850_c190214.nc
    Initial_conditions_dataset: finidat_interp_dest.nc
    PFT_physiological_constants_dataset: clm5_params.c171117.nc
    ltype_vegetated_or_bare_soil: 1
    ltype_crop: 2
    ltype_UNUSED: 3
    ltype_landice_multiple_elevation_classes: 4
    ltype_deep_lake: 5
    ltype_wetland: 6
    ltype_urban_tbd: 7
    ltype_urban_hd: 8
    ltype_urban_md: 9
    ctype_vegetated_or_bare

In [11]:
file_soilMoist_Sample = Dataset(ncFile_soilMoist_Sample, mode='r')
file_soilMoist_Sample
lons = file_soilMoist_Sample.variables['lon'][:]
lats = file_soilMoist_Sample.variables['lat'][:]
times = file_soilMoist_Sample.variables['time'][:]
ntim = times.shape[0]
print(ntim)
time = np.arange(ntim)
time
param_soilMoist_Sample = file_soilMoist_Sample.variables['SOILWATER_10CM'][:]
print(param_soilMoist_Sample.shape)
print('')

120
(120, 192, 288)



In [12]:
t,y,x = param_soilMoist_Sample.shape
print(y)
print(x)
print(t)
nDecades = 16
monStepHistTot = t*nDecades + 60   # =185001_201412 (60 is half of 120)
print(monStepHistTot)
ensMem = 50
print(ensMem)

192
288
120
1980
50


In [13]:
print(lats)

[-90.        -89.057594  -88.11518   -87.172775  -86.23037   -85.28796
 -84.34555   -83.403145  -82.46073   -81.518326  -80.57591   -79.63351
 -78.6911    -77.74869   -76.80628   -75.86388   -74.92146   -73.97906
 -73.03665   -72.09424   -71.15183   -70.20943   -69.26701   -68.32461
 -67.3822    -66.43979   -65.49738   -64.55498   -63.612564  -62.67016
 -61.72775   -60.78534   -59.842934  -58.900524  -57.958115  -57.015705
 -56.0733    -55.13089   -54.18848   -53.246075  -52.303665  -51.361256
 -50.41885   -49.47644   -48.53403   -47.59162   -46.649216  -45.706806
 -44.764397  -43.82199   -42.87958   -41.937172  -40.994766  -40.052357
 -39.109947  -38.167538  -37.225132  -36.282722  -35.340313  -34.397907
 -33.455498  -32.51309   -31.57068   -30.628273  -29.685863  -28.743456
 -27.801046  -26.858639  -25.916231  -24.973822  -24.031414  -23.089005
 -22.146597  -21.20419   -20.26178   -19.319372  -18.376963  -17.434555
 -16.492147  -15.549738  -14.607329  -13.664922  -12.722513  -11.7801

In [14]:
print(lons)

[  0.     1.25   2.5    3.75   5.     6.25   7.5    8.75  10.    11.25
  12.5   13.75  15.    16.25  17.5   18.75  20.    21.25  22.5   23.75
  25.    26.25  27.5   28.75  30.    31.25  32.5   33.75  35.    36.25
  37.5   38.75  40.    41.25  42.5   43.75  45.    46.25  47.5   48.75
  50.    51.25  52.5   53.75  55.    56.25  57.5   58.75  60.    61.25
  62.5   63.75  65.    66.25  67.5   68.75  70.    71.25  72.5   73.75
  75.    76.25  77.5   78.75  80.    81.25  82.5   83.75  85.    86.25
  87.5   88.75  90.    91.25  92.5   93.75  95.    96.25  97.5   98.75
 100.   101.25 102.5  103.75 105.   106.25 107.5  108.75 110.   111.25
 112.5  113.75 115.   116.25 117.5  118.75 120.   121.25 122.5  123.75
 125.   126.25 127.5  128.75 130.   131.25 132.5  133.75 135.   136.25
 137.5  138.75 140.   141.25 142.5  143.75 145.   146.25 147.5  148.75
 150.   151.25 152.5  153.75 155.   156.25 157.5  158.75 160.   161.25
 162.5  163.75 165.   166.25 167.5  168.75 170.   171.25 172.5  173.75
 175. 

## Setting US boundaries

In [15]:
# Save only U.S. 
latboundsUs = [ 25 , 55]
lonboundsUs = [230 , 300]

In [16]:
# latitude lower and upper index
latli = np.argmin( np.abs( lats - latboundsUs[0] ) )
latui = np.argmin( np.abs( lats - latboundsUs[1] ) ) 

# longitude lower and upper index
lonli = np.argmin( np.abs( lons - lonboundsUs[0] ) )
lonui = np.argmin( np.abs( lons - lonboundsUs[1] ) )  

In [17]:
print(latli, latui)

122 154


In [18]:
print(lonli, lonui)

184 240


In [19]:
print(param_soilMoist_Sample.shape)

(120, 192, 288)


In [20]:
param_soilMoist_Sample_US = param_soilMoist_Sample[:, latli:latui , lonli:lonui]
print(param_soilMoist_Sample_US.shape)

(120, 32, 56)


In [21]:
t,y_US,x_US = param_soilMoist_Sample_US.shape
print(y_US)
print(x_US)
print(t)

32
56
120


In [22]:
#
monSoilMoistUsHistEnsMem = np.zeros((monStepHistTot, y_US, x_US, ensMem, ))  

monSoilMoistUsHistEnsMem[:] = np.nan
print(monSoilMoistUsHistEnsMem.shape)

(1980, 32, 56, 50)


In [23]:

filePath = '/glade/campaign/cgd/cesm/CESM2-LE/lnd/proc/tseries/month_1/SOILWATER_10CM'

yrCtrlIds = ['1001', '1021', '1041', '1061', '1081', '1101', '1121', '1141', '1161', '1181', '1231', '1251', '1281', '1301'];
yrId = 0
monId = 0

monSt = 1
monEnd = 12

memId = -1
for yrCtrl in yrCtrlIds:    
    yrId = yrId + 1   
    
    if yrId < 11:
        memId = memId + 1
        tStepSt = 0
        yrHistSt = 1849
        yrHistEnd = yrHistSt + 9
    
        monId = monId + 1  
        yrMonDigits = str(yrCtrl)+'.' + str(monId).rjust(3,'0')
        print(yrMonDigits)
        
        for decade in range(17):
            yrHistSt = yrHistSt + 1
            yrHistEnd = yrHistEnd + 1
            
            yrMonHistSt = str(yrHistSt) + str(monSt).rjust(2,'0')
            yrMonHistEnd = str(yrHistEnd) + str(monEnd).rjust(2,'0')                      
            
            
            fileName = 'b.e21.BHISTcmip6.f09_g17.LE2-'+yrMonDigits+'.clm2.h0.SOILWATER_10CM.' +yrMonHistSt+'-'+yrMonHistEnd+'.nc'
            
            
            ncFile_soilMoist = str(filePath) + '/' + str(fileName)
            file_soilMoist = Dataset(ncFile_soilMoist, mode='r')                        
            param_soilMoistToAdd = file_soilMoist.variables['SOILWATER_10CM'][:, latli:latui , lonli:lonui]            
            t,y_US,x_US = param_soilMoistToAdd.shape
            tStepEnd = tStepSt + t            
           
                        
            monSoilMoistUsHistEnsMem[tStepSt:tStepEnd, :, :, memId] = param_soilMoistToAdd  
            
            yrHistSt = yrHistEnd
            tStepSt = tStepEnd
            
            if decade <15:
                yrHistEnd = yrHistSt + 9
            else:
                yrHistEnd = yrHistSt + 4
            
    else:         
    
        for monCont in range(1, 11): 
            
            tStepSt = 0
            memId = memId + 1
            yrMonDigits = str(yrCtrl)+'.' + str(monCont).rjust(3,'0')
            print(yrMonDigits)
            
            yrHistSt = 1849
            yrHistEnd = yrHistSt + 9
            
            for decade in range(17):            
                yrHistSt = yrHistSt + 1
                yrHistEnd = yrHistEnd + 1
    
                yrMonHistSt = str(yrHistSt) + str(monSt).rjust(2,'0')
                yrMonHistEnd = str(yrHistEnd) + str(monEnd).rjust(2,'0')  
                
                fileName = 'b.e21.BHISTcmip6.f09_g17.LE2-'+yrMonDigits+'.clm2.h0.SOILWATER_10CM.' +yrMonHistSt+'-'+yrMonHistEnd+'.nc'
                            
                ncFile_soilMoist = str(filePath) + '/' + str(fileName)
                file_soilMoist = Dataset(ncFile_soilMoist, mode='r')                        
                param_soilMoistToAdd = file_soilMoist.variables['SOILWATER_10CM'][:, latli:latui , lonli:lonui]              
                t,y_US,x_US = param_soilMoistToAdd.shape
                tStepEnd = tStepSt + t                
                                    
                monSoilMoistUsHistEnsMem[tStepSt:tStepEnd, :, :, memId] = param_soilMoistToAdd                 
            
                
                yrHistSt = yrHistEnd
                tStepSt = tStepEnd   
                
                if decade <15:
                    yrHistEnd = yrHistSt + 9
                else:
                    yrHistEnd = yrHistSt + 4

1001.001
1021.002
1041.003
1061.004
1081.005
1101.006
1121.007
1141.008
1161.009
1181.010
1231.001
1231.002
1231.003
1231.004
1231.005
1231.006
1231.007
1231.008
1231.009
1231.010
1251.001
1251.002
1251.003
1251.004
1251.005
1251.006
1251.007
1251.008
1251.009
1251.010
1281.001
1281.002
1281.003
1281.004
1281.005
1281.006
1281.007
1281.008
1281.009
1281.010
1301.001
1301.002
1301.003
1301.004
1301.005
1301.006
1301.007
1301.008
1301.009
1301.010


In [24]:
print(monSoilMoistUsHistEnsMem.shape)

(1980, 32, 56, 50)


In [25]:
monSoilMoistUsHistEnsMean = np.nanmean(monSoilMoistUsHistEnsMem, axis=3)
print(monSoilMoistUsHistEnsMean.shape)

(1980, 32, 56)


## Sub-setting data for 1930~2014

In [26]:
histYrSt  = 1850
histTrgtYrSt = 1930
histYrEnd = 2014
nYrsHist = (histYrEnd - histTrgtYrSt + 1)
print(nYrsHist)

85


In [27]:
histTrgtStepSt = (histTrgtYrSt - histYrSt)*12
histTrgtStepEnd = (histYrEnd - histYrSt + 1)*12
print(histTrgtStepSt, histTrgtStepEnd)

960 1980


In [28]:
print(monSoilMoistUsHistEnsMem.shape)

(1980, 32, 56, 50)


In [29]:
monSoilMoistUsHistSelEnsMembers = monSoilMoistUsHistEnsMem[histTrgtStepSt:histTrgtStepEnd,:,:,:]
print(monSoilMoistUsHistSelEnsMembers.shape)

(1020, 32, 56, 50)


In [30]:
monSoilMoistUsHistSelEnsMean = np.nanmean(monSoilMoistUsHistSelEnsMembers, axis = 3)
print(monSoilMoistUsHistSelEnsMean.shape)

(1020, 32, 56)


In [31]:
np.save('/glade/scratch/mrhaider/cesm2/param/soilMoist/mon/monSoilMoistUsHistSelEnsMembers.npy', monSoilMoistUsHistSelEnsMembers)

In [32]:
np.save('/glade/scratch/mrhaider/cesm2/param/soilMoist/mon/monSoilMoistUsHistSelEnsMean.npy', monSoilMoistUsHistSelEnsMean)

In [33]:
latsUs = lats[latli:latui]
lonsUs = lons[lonli:lonui]

print(latsUs)


[24.973822 25.916231 26.858639 27.801046 28.743456 29.685863 30.628273
 31.57068  32.51309  33.455498 34.397907 35.340313 36.282722 37.225132
 38.167538 39.109947 40.052357 40.994766 41.937172 42.87958  43.82199
 44.764397 45.706806 46.649216 47.59162  48.53403  49.47644  50.41885
 51.361256 52.303665 53.246075 54.18848 ]


In [34]:
latsUsArr = np.array(latsUs)
print(latsUsArr)

lonsUsArr = np.array(lonsUs)
print(lonsUsArr)


[24.973822 25.916231 26.858639 27.801046 28.743456 29.685863 30.628273
 31.57068  32.51309  33.455498 34.397907 35.340313 36.282722 37.225132
 38.167538 39.109947 40.052357 40.994766 41.937172 42.87958  43.82199
 44.764397 45.706806 46.649216 47.59162  48.53403  49.47644  50.41885
 51.361256 52.303665 53.246075 54.18848 ]
[230.   231.25 232.5  233.75 235.   236.25 237.5  238.75 240.   241.25
 242.5  243.75 245.   246.25 247.5  248.75 250.   251.25 252.5  253.75
 255.   256.25 257.5  258.75 260.   261.25 262.5  263.75 265.   266.25
 267.5  268.75 270.   271.25 272.5  273.75 275.   276.25 277.5  278.75
 280.   281.25 282.5  283.75 285.   286.25 287.5  288.75 290.   291.25
 292.5  293.75 295.   296.25 297.5  298.75]


In [36]:
np.save('/glade/scratch/mrhaider/cesm2/param/soilMoist/mon/latsUsArr.npy', latsUsArr)

In [37]:
np.save('/glade/scratch/mrhaider/cesm2/param/soilMoist/mon/lonsUsArr.npy', lonsUsArr)